### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6248,2023-11-21,Brasil Nbb,19:30,Fortaleza B.C.,Pinheiros,1.61,2.28,152.5,1.87,1.79,-4.5,2.08,1.67,YVsAF4uN,0.621118,0.438596,0.534759,0.558659,0.059715,0.0,0.0,NaN,NaN,260.834,80.369638,0.308126,207.044,129.709786,0.626484,0.000,0.000,154.77,428.56,0.0,0.0,0.0,0.0,0.243579,0.030912,0.154621,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6249,2023-11-21,Brasil Nbb,20:00,São José,Flamengo,3.84,1.25,154.5,1.89,1.87,7.5,2.07,1.67,v9r6GpQG,0.260417,0.800000,0.529101,0.534759,0.060417,0.0,0.0,NaN,NaN,156.748,62.681338,0.399886,NaN,NaN,NaN,0.000,0.000,263.98,91.12,0.0,0.0,0.0,1.0,0.719610,0.007522,0.151253,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6250,2023-11-21,Eua Nba,00:00,Golden State Warriors,Houston Rockets,1.35,3.65,217.5,1.86,2.07,-7.5,1.97,1.97,2TWMChVN,0.740741,0.273973,0.537634,0.483092,0.014713,0.8,0.2,NaN,NaN,227.294,39.578632,0.174130,287.160,48.667955,0.169480,207.884,261.712,258.33,317.20,0.0,0.0,0.0,0.0,0.650538,0.075569,0.000000,-5.00,-1.000,-0.350000,0.634559,0.3,-0.334559,-3.41,-0.682,-3.885630,0.342761,0.2,-0.142761
6251,2023-11-21,Eua Nba,21:00,Orlando Magic,Toronto Raptors,1.73,1.95,215.5,1.83,1.83,-3.5,2.01,1.68,nasMImT2,0.578035,0.512821,0.546448,0.546448,0.090855,0.4,0.6,NaN,NaN,219.408,36.305801,0.165472,242.454,92.316420,0.380758,214.022,259.008,279.65,365.76,0.0,0.0,0.0,0.0,0.084545,0.000000,0.126474,-1.01,-0.202,-3.613861,0.549640,0.6,0.050360,-0.45,-0.090,-10.555556,0.548032,0.4,-0.148032
6252,2023-11-21,Eua Nba,21:30,Atlanta Hawks,Indiana Pacers,1.51,2.31,250.5,1.87,1.79,-5.5,2.01,1.68,U365XzLs,0.662252,0.432900,0.534759,0.558659,0.095152,0.8,0.8,NaN,NaN,192.758,15.494141,0.080381,300.436,102.641933,0.341643,248.466,544.556,213.18,386.76,0.0,0.0,0.0,0.0,0.296170,0.030912,0.126474,-1.56,-0.312,-1.634615,0.568747,0.5,-0.068747,2.68,0.536,2.444030,0.253146,0.4,0.146854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6323,2023-11-21,Eua Ncaa,20:30,UNLV,Richmond,1.91,1.91,135.5,1.91,1.91,-1.5,1.95,1.71,UgeNpszh,0.523560,0.523560,0.523560,0.523560,0.047120,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,167.25,150.67,0.0,0.0,0.0,0.0,0.000000,0.000000,0.092735,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6324,2023-11-21,Eua Ncaa,21:30,Stephen F. Austin,Utah State,1.91,1.91,141.5,1.91,1.91,-2.5,2.10,1.63,bZnvsaZG,0.523560,0.523560,0.523560,0.523560,0.047120,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,176.30,180.84,0.0,0.0,0.0,0.0,0.000000,0.000000,0.178198,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6325,2023-11-21,Eua Ncaa,21:30,Wright State,Hofstra,2.18,1.73,157.5,1.80,1.95,1.5,2.00,1.69,KrlzruKA,0.458716,0.578035,0.555556,0.512821,0.036750,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,182.00,120.36,0.0,0.0,0.0,0.0,0.162761,0.056569,0.118809,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6326,2023-11-21,Eua Ncaa,22:00,Purdue,Tennessee,1.67,2.33,133.5,1.80,1.95,-4.5,2.05,1.71,jVA7PW3q,0.598802,0.429185,0.555556,0.512821,0.027987,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,87.98,78.11,0.0,0.0,1.0,1.0,0.233345,0.056569,0.127881,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Nba,Golden State Warriors,Houston Rockets,217.5,1.86,1.0000,Over
74,19:00,Eua Ncaa,High Point,Illinois State,146.5,1.91,0.8459,Over
